In [1]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from utils.helper_function import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

import optuna
import pickle
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas")
warnings.filterwarnings("ignore", category=UserWarning)


/tmp/ipykernel_24803/240062032.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/seg0fied/miniconda3/envs/biopkd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_sk = pd.read_csv("../data/skin-sensitization/train.csv")
test_sk = pd.read_csv("../data/skin-sensitization/val.csv")

In [3]:
train_sk.head()

,SMILES,Activity
0,CC(=O)CC(=O)NC1=CC(Cl)=CC=C1OC,0
1,CC1=NN2C(=NC3C=CC=CC=3C2=O)C1N=NC1C=CC(Cl)=CC=...,0
2,CCCCCCCCCCCCCCCCCCN(CCCCCCCCCCCCCCCCCC)C(=O)C1...,0
3,ClC1C=C(C=CC=1)OC1C=CC(Cl)=CC=1,0
4,OCCN1CCN(CCS(O)(=O)=O)CC1,0


In [4]:
test_sk.head()

,SMILES,Activity
0,CCC1(CO)COCOC1,0
1,CC(=O)C(=CC1C=CC=CC=1[N+]([O-])=O)C(=O)OC,1
2,[O-][N+](=O)C1C(=NN2C=CC=CC2=1)OCCO,0
3,CCCCCCCCCCCC(=O)OC1CC(C)(C)NC(C)(C)C1,0
4,CC(C)(C)CC(C)CC(=O)OOC(C)(C)C,1


In [5]:
# X,y = train_sk["SMILES"], train_sk['Activity']

smilesTrain = train_sk["SMILES"].to_numpy()
activityTrain = train_sk["Activity"].to_numpy()

smilesTest = test_sk["SMILES"].to_numpy()
activityTest = test_sk["Activity"].to_numpy()

In [6]:
idx = valid_indices(smilesTrain)
idx2 = valid_indices(smilesTest)

smilesTrain = smilesTrain[idx]
smilesTest = smilesTest[idx2]

[16:29:37] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[16:29:37] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 9 10 11 13 14 15


In [7]:
maccs_train = maccs_fp(smilesTrain)
maccs_test = maccs_fp(smilesTest)


In [8]:
activityTrain = activityTrain[idx]
activityTest = activityTest[idx2]

In [9]:
maccs_train = np.array(maccs_train) 
maccs_test = np.array(maccs_test)



In [10]:
def svm_modl(trial, xtrain, ytrain):
    n = trial.suggest_int('degree', 2, 5)
    model_svm = SVC(degree= n)

    scores = cross_validate(model_svm, xtrain, ytrain, cv=5, scoring='roc_auc')
    mean_roc = scores['test_score'].mean()

    return 1/(mean_roc + 1e-6)

In [11]:
study2 = optuna.create_study(study_name='SVM_study_skin_sensitization', storage= None)  # Create a new study.
study2.optimize(lambda trial: svm_modl(trial, maccs_train, activityTrain), n_trials=20)  # Invoke optimization of the objective function.

[I 2024-01-31 16:32:58,097] A new study created in memory with name: SVM_study_skin_sensitization
[I 2024-01-31 16:33:03,503] Trial 0 finished with value: 1.3692018388947174 and parameters: {'degree': 5}. Best is trial 0 with value: 1.3692018388947174.
[I 2024-01-31 16:33:07,878] Trial 1 finished with value: 1.3692018388947174 and parameters: {'degree': 4}. Best is trial 0 with value: 1.3692018388947174.
[I 2024-01-31 16:33:12,166] Trial 2 finished with value: 1.3692018388947174 and parameters: {'degree': 2}. Best is trial 0 with value: 1.3692018388947174.
[I 2024-01-31 16:33:16,452] Trial 3 finished with value: 1.3692018388947174 and parameters: {'degree': 3}. Best is trial 0 with value: 1.3692018388947174.
[I 2024-01-31 16:33:20,522] Trial 4 finished with value: 1.3692018388947174 and parameters: {'degree': 4}. Best is trial 0 with value: 1.3692018388947174.
[I 2024-01-31 16:33:24,745] Trial 5 finished with value: 1.3692018388947174 and parameters: {'degree': 5}. Best is trial 0 with

In [12]:
best_model2 = SVC(degree= study2.best_params['degree'])
best_model2.fit(maccs_train, activityTrain)

SVC(degree=5)

In [13]:
y_pred = best_model2.predict(maccs_test)
accuracy2 = accuracy_score(y_true= activityTest, y_pred= y_pred)
print(accuracy2)

0.6824966078697422


In [14]:
tn, fp, fn, tp = confusion_matrix(y_pred, activityTest).ravel()

ACC = (tp + tn)/(tp + tn + fn + fp)
SEN = tp/(tp + fn)
SPE = tn/(tn + fp)

print(f'Accuracy = {ACC}')
print(f'Sensitivity = {SEN}')
print(f'Specificity = {SPE}')


Accuracy = 0.6824966078697422
Sensitivity = 0.6857142857142857
Specificity = 0.6773049645390071


In [15]:
model_name = f"SVM_MACCS_model_Skin_sensitization{datetime.datetime.now().isoformat()}"
model_path = "../models/"
file = os.path.join(model_path, model_name)
pickle.dump(best_model2, open(file, 'wb'))